In [1]:
from ema_workbench import ema_logging, save_results, load_results
import pandas as pd
import os

C:\Users\cschott\AppData\Roaming\Python\Python311\site-packages\ema_workbench\em_framework\evaluators.py:58: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


The use of a full factorial design for scenario generation, while comprehensive, inevitably results in a subset of parameter combinations that are implausible or infeasible under real-world conditions. To maintain the analytical integrity and interpretability of the results, these non-viable scenarios must be systematically identified and excluded from the dataset.

#### I Exclusion of Cases with High Nourishment Frequency and Volume but Low Dry Bulk Density

Sand nourishments are known to elevate the concentration of suspended mineral sediments in the water column, thereby promoting mineral sediment deposition. In scenarios characterized by frequent nourishment interventions and high volumes of sand input, it is implausible for the resulting sediment deposits to exhibit low dry bulk density values $\rho=400$. Such combinations likely reflect unrealistic or inconsistent model behavior. Therefore, cases exhibiting both high nourishment frequency and quantity $C_{\mathcal{N}}$, coupled with low dry bulk density, are excluded from the analysis to ensure the physical plausibility and integrity of the modeled outcomes.

The following combinations of parameters are excluded due to their physical implausibility, as described above:

| N. concentration           | N. frequency             | Dry bulk density          |
|:----------------------------:|:-----------------------------:|:-------------------------:|
| 0.3                         | 1                              | 400                       |
| 0.5                         | 1                              | 400                       |
| 0.5                         | 5                              | 400                       |

#### II Exclusion of Cases with Implausible Sediment Deposition Fractions and Vegetation States

The parameter $f_d$ represents the fraction of sediment retained and reflects both vegetation-mediated trapping and management interventions. On mudflats ($E_0=0.4$), high values of $f_d \in (0.6, 0.8)$ imply strong sediment trapping, which is unrealistic for unvegetated or sparsely vegetated areas, and are therefore excluded. In the pioneer zone ($E_0=0.7$), extreme values ($f_d=0.2$ and $f_d=0.8$) are removed, as they do not plausibly reflect vegetation structure under either conservation or restoration conditions.

For the low marsh ($E_0=1.2$) and high marsh ($E_0=1.7$), values of $f_d=0.4$, $f_d=0.4$, and $f_d=0.8$ are retained, corresponding to management strategies such as no restoration, conservation, and active restoration in the low marsh, and excessive grazing, grazing, and no grazing in the high marsh. In both zones, $f_d=0.2$ is excluded due to incompatibility with the expected sediment trapping under vegetated conditions. These constraints help ensure the physical plausibility of modeled scenarios across the elevation gradient.

The following combinations of parameters are excluded due to their physical implausibility, as described above:
| Elevation                    | Depositing fraction           |
|:----------------------------:|:-----------------------------:|
| 0.4                          | 0.6, 0.8                      | 
| 0.7                          | 0.2, 0.8                      | 
| 1.2                          | 0.2                           | 
| 1.7                          | 0.2                           | 

####  III Exclusion of Cases with Nourishment Frequency but No Input

Cases with a non-zero nourishment frequency but zero nourishment amount ($C_{\mathcal{N}} = 0$) are excluded, as they imply sediment input events without actual material added—an implausible scenario.

The following combinations of parameters are excluded due to their physical implausibility, as described above:
| N. concentration             | N. frequency          |
|:----------------------------:|:-----------------------------:|
| 0.0                          | 5                    | 
| 0.0                          |10                     | 


#### 1. Define filter code

In [2]:
def apply_filters(df):
    filter_I = [
        (0.3, 1, 400),
        (0.5, 1, 400),
        (0.5, 5, 400),
    ]
    mask_I = df[['c_flood_nourishment', 'nourishment_frequency', 'rho_deposit']].apply(tuple, axis=1).isin(filter_I)
    df_I = df[~mask_I]

    filter_II = [
        (0.4, 0.6),
        (0.4, 0.8),
        (0.7, 0.2),
        (0.7, 0.8),
        (1.2, 0.2),
        (1.7, 0.2),
    ]
    mask_II = df_I[['z_init', 'fd']].apply(tuple, axis=1).isin(filter_II)
    df_II = df_I[~mask_II]

    filter_III = [
        (0.0, 10),
        (0.0, 5),
    ]
    mask_III = df_II[['c_flood_nourishment', 'nourishment_frequency']].apply(tuple, axis=1).isin(filter_III)
    return df_II[~mask_III]

#### 2. Load dataset

In [3]:
# RCP 8.5 dataset
#experiments, outcomes=load_results('model_output_M/model_output_raw.tar.gz')
experiments, outcomes=load_results('model_output_raw.tar.gz')
outcomes = pd.DataFrame(outcomes)
out_raw=pd.concat([experiments, outcomes], axis = 1)

#### 3. Apply filter and store filtered dataset

In [5]:
# Apply filters and store results with names
filtered_dataset = apply_filters(out_raw) 

# Store in predefined output folder
filtered_dataset.to_csv("model_output_clean.txt", sep='\t', index=False)

In [6]:
filtered_dataset

,c_flood,c_flood_nourishment,fd,nourishment_frequency,rcp,rho_deposit,s_subsidence,site,slr_select,z_init,scenario,policy,model,crit_year,growth_total,slope_norm_10,est_time,est_crit_year
0,0.05,0.0,0.2,1,rcp26,400,0.0027,S15,1,0.4,4,NaN,marshaccretion,2101,0.171807,-9.898503e-03,98.660792,2199.660792
4,0.05,0.0,0.2,1,rcp26,400,0.0027,S15,2,0.4,8,NaN,marshaccretion,2101,0.082943,-1.916339e-02,0.280064,2101.280064
8,0.05,0.0,0.2,1,rcp26,400,0.0027,S15,3,0.4,12,NaN,marshaccretion,2058,-0.011490,2.220446e-16,17.000000,2058.000000
12,0.05,0.0,0.2,1,rcp26,400,0.0027,S33,1,0.4,16,NaN,marshaccretion,2101,0.377851,-1.682433e-03,964.912963,3065.912963
16,0.05,0.0,0.2,1,rcp26,400,0.0027,S33,2,0.4,20,NaN,marshaccretion,2101,0.289847,-7.383435e-03,127.420968,2228.420968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46647,0.20,0.5,0.8,10,rcp85,1200,0.0050,S33,1,1.7,46651,NaN,marshaccretion,2101,-0.048469,-5.145372e-03,143.501122,2244.501122
46650,0.20,0.5,0.8,10,rcp85,1200,0.0050,S33,2,1.2,46654,NaN,marshaccretion,2101,0.350695,-1.439202e-02,50.063334,2151.063334
46651,0.20,0.5,0.8,10,rcp85,1200,0.0050,S33,2,1.7,46655,NaN,marshaccretion,2101,-0.133335,-1.061966e-02,48.034785,2149.034785
46654,0.20,0.5,0.8,10,rcp85,1200,0.0050,S33,3,1.2,46658,NaN,marshaccretion,2101,0.254902,-2.604880e-02,11.261905,2112.261905
